# Amazon SageMaker + WhyLabs

In [ ]:
import json
import boto3
from dotenv import dotenv_values
from utils import delete_model, delete_endpoint_config, delete_endpoint, is_endpoint_running

## AWS configuration

In [ ]:
AWS_PROFILE_NAME = "mfa"
AWS_REGION_NAME = session.region_name

In [ ]:
session = boto3.session.Session(profile_name=AWS_PROFILE_NAME)
sts = session.client("sts")
sm = session.client('sagemaker', region_name=AWS_REGION_NAME)
AWS_ACCOUNT_ID = sts.get_caller_identity().get("Account")
DOCKER_IMAGE_NAME = "whylabs-sagemaker"

## Custom image building and pushing to ECR

In [3]:
!./build_push.sh $DOCKER_IMAGE_NAME $AWS_PROFILE_NAME

Image name whylabs-sagemaker
Profile name default

An error occurred (ExpiredToken) when calling the GetCallerIdentity operation: The security token included in the request is expired


## Create SageMaker Endpoint

The steps to deploy a SageMaker model are:

1. Create a model
2. Create an endpoint configuration
3. Create a SageMaker endpoint

### 1. Model Creation

In [ ]:
ECR_IMAGE_URI = f"{AWS_ACCOUNT_ID}.dkr.ecr.{AWS_REGION_NAME}.amazonaws.com/{DOCKER_IMAGE_NAME}:latest"
ENDPOINT_NAME = "whylabs-sagemaker"
EXECUTION_ROLE_ARN = f"arn:aws:iam::{AWS_ACCOUNT_ID}:role/SageMakerExecution"
INSTANCE_TYPE = "ml.m4.xlarge"

Load variables important for __WhyLabs configuration__ defined inside __.env file__ as dictionary. This values will be settled once the docker container is running within SageMaker.

In [ ]:
# Load .env file as dictionary
environment = dotenv_values("code/.env")

In [ ]:
# ECR image to be used
PRIMARY_CONTAINER = {
    'Image': ECR_IMAGE_URI, 
    "Environment": environment,
}

In [ ]:
try:
    # Create sagemaker model
    r = sm.create_model(
        ModelName=ENDPOINT_NAME,
        ExecutionRoleArn=EXECUTION_ROLE_ARN,
        PrimaryContainer=PRIMARY_CONTAINER,
    )
    print("SageMaker model created.")
except Exception as e:
    print(e.response["Error"])

### 2. Endpoint Config creation

In [ ]:
ENDPOINT_CONFIG_NAME = ENDPOINT_NAME + '-config'

In [ ]:
try:
    # create endpoint configuration
    _ = sm.create_endpoint_config(
        EndpointConfigName=ENDPOINT_CONFIG_NAME,
        ProductionVariants=[
            {
                'InstanceType': INSTANCE_TYPE,
                'InitialVariantWeight': 1,
                'InitialInstanceCount': 1,
                'ModelName': ENDPOINT_NAME,
                'VariantName': 'AllTraffic'
            }
        ]
    )
    print("Endpoint configuration created.")
except Exception as e:
    print(e.response["Error"])

### 3. Endpoint creation

In [ ]:
try:
    # create endpoint
    r = sm.create_endpoint(
        EndpointName=ENDPOINT_NAME,
        EndpointConfigName=ENDPOINT_CONFIG_NAME
    )
    print(f"Completed {endpoint_name} model endpoint deployment !!!")
except Exception as e:
    print(e.response["Error"])

## Test Endpoint 

In [ ]:
# Payload for /invocations endpoint
payload = json.dumps({
    "sepal_length_cm": 5.1,
    "sepal_width_cm": 3.5,
    "petal_length_cm": 1.4,
    "petal_width_cm": 0.2
})

In [ ]:
# Invoke the endpoint using
sg = session.client("runtime.sagemaker", region_name=AWS_REGION_NAME)
status = is_endpoint_running(ENDPOINT_NAME, AWS_PROFILE_NAME, AWS_REGION_NAME)
# Check if model was created successfully
if status == "InService":
    response = sg.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        Body=payload,
        ContentType='application/json',
    )
    # Decode the response
    print(json.loads(response["Body"].read().decode("utf-8")))
else:
    print(f"Endpoint status is {status}")

## Delete AWS resources

In [ ]:
status = is_endpoint_running(ENDPOINT_NAME, AWS_PROFILE_NAME, AWS_REGION_NAME)

In [ ]:
if status == "InService":
    delete_model(sm, ENDPOINT_NAME)
    delete_endpoint_config(sm, ENDPOINT_CONFIG_NAME)
    delete_endpoint(sm, ENDPOINT_NAME)